In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
from datetime import datetime

In [54]:
#database connection settings
import psycopg2

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)

In [17]:
################################################################################
########## WIKIPEDIA LIST OF MUSICAL WORKS WITH COMPOSER AND LOAD TO DB ########
################################################################################
from SPARQLWrapper import SPARQLWrapper, JSON

import rdflib
from rdflib.graph import Graph
from rdflib.namespace import Namespace
from rdflib import plugin

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

# create a construct query template.
# NOTE: To keep it simple we create an exact copy matching a query in the local graph.
sparql.setQuery("""
SELECT * WHERE {
  {
  SELECT ?composition ?compositionLabel ?composer ?composerLabel ?workType
  WHERE {
     {
      ?composition wdt:P31 wd:Q1344.  
      OPTIONAL { ?composition wdt:P86 ?composer. }
      BIND ("opera" AS ?workType)
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en,ca,es,fi,de,sv,sl,it,fr,nl,es,ru,hu,no,cs,ca,sv,bg,als,ja,pt,az,et,be,uk". }    
      }
    UNION
    {
      ?composition wdt:P31 wd:Q170384.  
      OPTIONAL { ?composition wdt:P86 ?composer. }
      BIND ("operetta" AS ?workType)    
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en,ca,es,fi,de,sv,sl,it,fr,nl,es,ru,hu,no,cs,ca,sv,bg,als,ja,pt,az,et,be,uk". }    
      }  
    UNION
    {
      ?composition wdt:P31 wd:Q17851193.  
      OPTIONAL { ?composition wdt:P86 ?composer. }
      BIND ("opera cycle" AS ?workType)    
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en,ca,es,fi,de,sv,sl,it,fr,nl,es,ru,hu,no,cs,ca,sv,bg,als,ja,pt,az,et,be,uk". }    
      }  
    UNION    
    {
      ?composition wdt:P31 wd:Q2743.  
      OPTIONAL { ?composition wdt:P86 ?composer. }
      BIND ("musical" AS ?workType)    
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en,ca,es,fi,de,sv,sl,it,fr,nl,es,ru,hu,no,cs,ca,sv,bg,als,ja,pt,az,et,be,uk". }    
      }  
    UNION       
    {
      ?composition wdt:P31 wd:Q85477.  
      OPTIONAL { ?composition wdt:P86 ?composer. }
      BIND ("oratorio" AS ?workType)        
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en,ca,es,fi,de,sv,sl,it,fr,nl,es,ru,hu,no,cs,ca,sv,bg,als,ja,pt,az,et,be,uk". }    
      }    
    UNION
    {
      ?composition wdt:P31 wd:Q174873.  
      OPTIONAL { ?composition wdt:P86 ?composer. }
      BIND ("cantata" AS ?workType)        
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en,ca,es,fi,de,sv,sl,it,fr,nl,es,ru,hu,no,cs,ca,sv,bg,als,ja,pt,az,et,be,uk". }    
      }      
    #FILTER(!bound(?composer))
  }
} 
  #FILTER regex(?compositionLabel, "^Sw.")
}  
ORDER BY ?compositionLabel
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

musical_works = []
results = results["results"]["bindings"]

for result in results:
    musical_work = []
    work          = result['compositionLabel']['value']    
    work_url      = result['composition']['value']
    work_uri      = work_url.split('/')[-1]
    work_type     = result['workType']['value']
    
    try:
        composer      = result['composerLabel']['value']
        composer_url  = result['composer']['value']    
        composer_uri  = composer_url.split('/')[-1]
    except:
        composer      = ''
        composer_url  = ''
        composer_uri  = ''        

    musical_work.append(work)
    musical_work.append(work_url) 
    musical_work.append(work_uri)         
    musical_work.append(work_type)
    musical_work.append(composer)
    musical_work.append(composer_url)        
    musical_work.append(composer_uri)            
    musical_works.append(musical_work)

In [18]:
for mw in musical_works:
    cursor = conn.cursor()    
    update = "UPDATE WORKS w SET work_type = '" + mw[3] + "' WHERE uri = '" + mw[2] + "'; COMMIT;"
    cursor.execute(update)
    
    if mw[4]!='':
        insert = 'INSERT INTO WORKS (NAME, URL, URI, COMPOSER_URI, WORK_TYPE) VALUES ('
        values = "'" + mw[0].replace("'", "''") + "', '" + mw[1] + "', '" + mw[2] + "', '" + mw[6] + "', '" + mw[3] + "'); "
    else:
        insert = 'INSERT INTO WORKS (NAME, URL, URI, WORK_TYPE) VALUES ('
        values = "'" + mw[0].replace("'", "''") + "', '" + mw[1] + "', '" + mw[2] + "', '" + mw[3] + "'); "        
    commit = 'COMMIT;'    
    
    try:
        cursor = conn.cursor()    
        cursor.execute(insert + values + commit)
    except:
        None

In [19]:
len(musical_works)

8971

In [58]:
query = "SELECT DISTINCT composer_uri FROM works WHERE composer_id IS NULL AND composer_uri IS NOT NULL"
cursor.execute(query)
mc_df = pd.DataFrame(cursor.fetchall(), columns=['composer_uri'])

from wikidata.client import Client
client = Client()
cursor = conn.cursor()    

for idx,row in mc_df.iterrows():
    entity = client.get(row['composer_uri'], load=True)
    name = str(entity.label)
    url = 'https://www.wikidata.org/wiki/' + row['composer_uri']
    
    insert = 'INSERT INTO COMPOSERS (NAME, URL, URI) VALUES ('
    values = "'" + name.replace("'", "''") + "', '" + url + "', '" + row['composer_uri'] + "'); "
    commit = 'COMMIT;'    
#     print(insert + values + commit)    
    cursor.execute(insert + values + commit)
    
#     print(entity.label)

# gender_prop = client.get('P21')
# gender = entity[gender_prop]

# print(gender_prop)
# print(entity[gender_prop])

In [71]:
from wikidata.client import Client
client = Client()
cursor = conn.cursor()    
entity = client.get('Q20145', load=True)
name = str(entity.label)

for k in list(entity):
    print(k)
    print(k.label)
#     k_prop = client.get(k)
#     print(k_prop.label)    
#     k_pr = entity[k_prop]
#     print(k_pr)
    
#     print(entity.label)

# gender_prop = client.get('P21')
# gender = entity[gender_prop]

# print(gender_prop)
# print(entity[gender_prop])

<wikidata.entity.Entity P1477>
birth name
<wikidata.entity.Entity P21>
sex or gender
<wikidata.entity.Entity P358>
discography
<wikidata.entity.Entity P373>
Commons category
<wikidata.entity.Entity P106>
occupation
<wikidata.entity.Entity P434>
MusicBrainz artist ID
<wikidata.entity.Entity P345>
IMDb ID
<wikidata.entity.Entity P31>
instance of
<wikidata.entity.Entity P856>
official website
<wikidata.entity.Entity P569>
date of birth
<wikidata.entity.Entity P18>
image
<wikidata.entity.Entity P646>
Freebase ID
<wikidata.entity.Entity P27>
country of citizenship
<wikidata.entity.Entity P412>
voice type
<wikidata.entity.Entity P734>
family name
<wikidata.entity.Entity P1303>
instrument
<wikidata.entity.Entity P136>
genre
<wikidata.entity.Entity P910>
topic's main category
<wikidata.entity.Entity P214>
VIAF ID
<wikidata.entity.Entity P1953>
Discogs artist ID
<wikidata.entity.Entity P2002>
Twitter username
<wikidata.entity.Entity P1649>
KMDb person ID
<wikidata.entity.Entity P2003>
Instagram